In [111]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('news.csv')
df.dropna(inplace=True)
df.reset_index(inplace=True)
train, test = train_test_split(df, test_size=0.2)
train.reset_index(inplace=True)
test.reset_index(inplace=True)
#df['content'].value_counts()

In [112]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')

In [113]:
import string
from num2words import num2words
from nltk.stem import WordNetLemmatizer
def stopword_fun(x):
    words = [word for word in x['content'].split() if word.lower() not in sw_nltk]
    text = " ".join(words)
    text = "".join([i for i in text if i not in string.punctuation])
    words = [WordNetLemmatizer().lemmatize(word) for word in text.split(' ')]
    text = ''
    for word in words:
        
        if word.isnumeric():
            text  = text + " " +num2words(float(word))
        elif len(word) >2:
            text =  text + " " + word
    words = [word for word in text.split() if word.lower() not in sw_nltk]    
    text = " ".join(words)
    text = "".join([i for i in text if i not in string.punctuation])
    text = text.lower()
    return text
def stopword_fun_title(x):
    words = [word for word in x['title'].split() if word.lower() not in sw_nltk]
    text = " ".join(words)
    text = "".join([i for i in text if i not in string.punctuation])
    words = [WordNetLemmatizer().lemmatize(word) for word in text.split(' ')]
    text = ''
    for word in words:
        
        if word.isnumeric():
            text  = text + " " +num2words(float(word))
        elif len(word) >2:
            text =  text + " " + word
    words = [word for word in text.split() if word.lower() not in sw_nltk]    
    text = " ".join(words)
    text = "".join([i for i in text if i not in string.punctuation])
    text = text.lower()
    return text
def preprocess_query(x):
    words = [word for word in x.split() if word.lower() not in sw_nltk]
    text = " ".join(words)
    text = "".join([i for i in text if i not in string.punctuation])
    words = [WordNetLemmatizer().lemmatize(word) for word in text.split(' ')]
    text = ''
    for word in words:
        
        if word.isnumeric():
            text  = text + " " +num2words(float(word))
        elif len(word) >2:
            text =  text + " " + word
    words = [word for word in text.split() if word.lower() not in sw_nltk]    
    text = " ".join(words)
    text = "".join([i for i in text if i not in string.punctuation])
    text = text.lower()
    return text
content = train.apply(stopword_fun,axis=1)

title = train.apply(stopword_fun_title,axis=1)



In [114]:
print(content[1])
df['content'][1]

bts leader aka kim namjoons album indigo going december two first solo album past released two mixtapes work mono got lot critical appreciation working indigo quite time described archive twenty bts army feeling emotional affair aka kim namjoon pouring pain joy fear beautiful lyric well look like keeping promise promotion indigo fans remember said would like meet armys small group maybe coffee chat look like finally happening big hit music announced hold concert two hundred armys seems batch fifty people selected via raffle system people selected must purchased album via weverse also minor allowed concert recorded per broadcasting law minor filmed unless accompanied parent korea minor allowed clause ignited imagination bts army daddy joon take look tweet day back namjoon went viral tiktok hot performance sexy nukim song balming tiger featured small rap verse namjoon aka right official schedule paid visit museum enjoying new york fullest


'RM aka Kim Namjoon was the first member to join BTS. The group released their debut single album 2 Cool 4 Skool on June 12, 2013. Apart from RM, BTS also features Jin, Suga, J-Hope, Jimin, V, and Jungkook. In an interview with Hypebeast, RM said, "This is the most difficult question to answer truthfully. Recently, I watched Everything Everywhere All At Once. That film visualized many of the ideas that I’ve had, such as the idea about multiple versions of myself existing based on small choices I made. I often think about what it would have been like if I continued my studies or became something other than a musician." "To be honest, one decision that I had often thought about was my choice to become a part of a boy band. In the late 2000s, musicians like Zico, Changmo, and Giriboy were the people that I started out with. In my journey with BTS, I drifted further and further away from that world and was tormented by the thought that the people that I liked – and the people who enjoyed t

In [115]:
from nltk.tokenize import word_tokenize
processed_text = []
processed_title = []
print(len(content),len(title))

for j,i in enumerate(content):
   
    processed_text.append(word_tokenize(i))
    processed_title.append(word_tokenize(title[j]))


644 644


In [116]:
DF = {}
from collections import Counter
for i in range(len(content)):
    tokens = processed_text[i]
    for w in tokens:
        if w in DF.keys():
            DF[w].add(i)
        else:
            DF[w] = {i}
   
    tokens = processed_title[i]
    for w in tokens:
        if w in DF.keys():
            DF[w].add(i)
        else:
            DF[w] = {i}

for i in DF:
    DF[i] = len(DF[i])
total_vocab_size = len(DF)

total_vocab = [x for x in DF]
def doc_freq(word):
    c = 0
    if word in DF.keys():
        c = DF[word]
    else:
        pass
    return c


In [117]:

doc = 0

tf_idf = {}
N = len(content)

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df_val = doc_freq(token)
        idf = np.log((N+1)/(df_val+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1
cnt = 0
for i in tf_idf.values():
    if i == 0:
        cnt = cnt +1
cnt

0

In [118]:
doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df_val = doc_freq(token)
        idf = np.log((N+1)/(df_val+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1
len(tf_idf_title)

8434

In [119]:
alpha = 0.7
for i in tf_idf:
    tf_idf[i] *= alpha
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]


In [120]:
def matching_score(k, query):
    preprocessed_query = preprocess_query(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:k]:
        l.append(i[0])
    
    print(l)
matching_score(10,test['content'][0])


Matching Score

Query: Speaking about the friendship tattoo, V said as translated by Twitter user @BTStranslation_, "I still... haven't gotten the friendship ‘7' tattoo. I haven't been able to decide where to get it so I'm trying to decide quickly. Will you guys help me decide? I'll go get it done with Jungkook once I choose where I'll get it done." When a fan suggested that he should get the tattoo under the left toe, V said, "I also thought about it." Recently, RM revealed that he got the tattoo inked on his leg while eagle-eyed fans discovered that Jimin got it on his finger. V spoke about Jungkook's new song My You, released ahead of BTS' ninth anniversary, and said, "Jungkook's song? I heard Jungkook's song about a month ago... it's good, isn't it? Shall we listen to his song once? He made this song well!!" He also sang My You and later Jimin's OST debut song, With You, from Our Blues. He also said, "Our Blues…to be honest, from the 13th, 14th episode I’ve been crying at the end o

In [121]:
import math
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q


In [122]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess_query(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
       
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)
cosine_similarity(10,test['content'][0])

Cosine Similarity

Query: Speaking about the friendship tattoo, V said as translated by Twitter user @BTStranslation_, "I still... haven't gotten the friendship ‘7' tattoo. I haven't been able to decide where to get it so I'm trying to decide quickly. Will you guys help me decide? I'll go get it done with Jungkook once I choose where I'll get it done." When a fan suggested that he should get the tattoo under the left toe, V said, "I also thought about it." Recently, RM revealed that he got the tattoo inked on his leg while eagle-eyed fans discovered that Jimin got it on his finger. V spoke about Jungkook's new song My You, released ahead of BTS' ninth anniversary, and said, "Jungkook's song? I heard Jungkook's song about a month ago... it's good, isn't it? Shall we listen to his song once? He made this song well!!" He also sang My You and later Jimin's OST debut song, With You, from Our Blues. He also said, "Our Blues…to be honest, from the 13th, 14th episode I’ve been crying at the en

In [123]:
content[525]

'took time figure font position tattoo process getting inked took two minute interaction fan said english “it took two hour stamp body check font position took two damn minute actual tattoo body two line one line ouch 2nd line ouch done” previously shared glimpse friendship tattoo fan seems bts leader’s tribute sevenmember band seven also name bts’ two thousand twenty album member discussed getting inked several occasion jhope jimin jin also similar tattoo talking tattoo earlier said weverse “we’ve talking friendship tattoo forever we’ll someday please look forward it” also mentioned member discussion getting friendship tattoo let’s tattoo seven dot seven something simply tattoo number seven we’re actually want ankle know it’s simple” said currently bts member jin suga jhope jimin jungkook break focus individual career goal announcing hiatus said enjoyed bts group member feel exhausted know kind group last album proof released last month'